# regrid the dataset to a healpix grid

Since the default geographic rectilinear grid has non-uniform grid sizes and distances and is thus not suitable for this application, we have to transform the dataset to one with equal distances between the cell centers. The Healpix cells satisfy these conditions, and can be used for spatial convolutions when reshaped to a 2D array according to the "nested" cell numbering.

As with any interpolation, the linear interpolation supported by the `healpy` library is split into the computation of weights and the application of those weights to the data.

In [1]:
import xarray as xr
from pangeo_fish.healpy import create_grid, HealpyRegridder

parameters

In [2]:
nside: int = 4096  # healpix resolution
rot = {"lat": 0, "lon": 30}
input_path: str
output_path: str

scheduler_address: str = None

In [3]:
tag_url: str

catalog: str
catalog_parameters: dict = {}

scheduler_address: str | None = None

relative_depth_threshold: float = 0.8

diff_path: str

# local PC
tag_url="/Users/todaka/python/git/pangeo-fish/data_local/fish-intel/tag/nc/A18832.nc"
catalog = "https://data-taos.ifremer.fr/kerchunk/ref-copernicus.yaml"
scheduler_address: str | None = None
catalog = "https://data-taos.ifremer.fr/kerchunk/ref-marc.yaml"
diff_path="/Users/todaka/python/git/pangeo-fish/data_local/fish-intel/A18832-f1_e2500/diff.zarr"
# mars
catalog = "/home/datawork-taos-s/intranet/kerchunk/ref-marc.yaml"
catalog_parameters: dict = {  "region": "f1_e2500",  "year": "2022"}
diff_path="/home/datawork-taos-s/public/fish/A18832-f1_e2500/diff.zarr"
diff_healpix_path="/home/datawork-taos-s/public/fish/A18832-f1_e2500/diff_healpix.zarr"

# Datarmor
tag_url="/home/datawork-lops-iaocea/data/fish-intel/tag/nc/A18832.nc"
# copernicus
catalog="/home/datawork-taos-s/intranet/kerchunk/ref-copernicus.yaml"
#catalog_parameters: dict = {  "type": ["2022_3D","2022_2D","mdt"]}
diff_path="/home/datawork-taos-s/public/fish/A18832-copernicus/diff.zarr"
diff_healpix_path="/home/datawork-taos-s/public/fish/A18832-copernicus/diff_healpix.zarr"







cluster

In [4]:
import dask_hpcconfig
from distributed import Client

In [5]:
cluster = dask_hpcconfig.cluster("datarmor-local")
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 56,Total memory: 120.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45207,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 56
Started: Just now,Total memory: 120.00 GiB
Comm: tcp://127.0.0.1:50632,Total threads: 7
Dashboard: http://127.0.0.1:52623/status,Memory: 15.00 GiB
Nanny: tcp://127.0.0.1:47534,


## read the data

In [6]:
ds = xr.open_dataset(diff_path, engine="zarr", chunks={})
ds

<xarray.Dataset>
Dimensions:    (lat: 371, lon: 264, time: 1814)
Coordinates:
  * lat        (lat) float32 46.0 46.01 46.03 46.04 ... 50.96 50.97 50.99 51.0
    latitude   (lat, lon) float32 dask.array<chunksize=(371, 264), meta=np.ndarray>
  * lon        (lon) float32 -8.0 -7.97 -7.939 ... -0.09091 -0.06061 -0.0303
    longitude  (lat, lon) float32 dask.array<chunksize=(371, 264), meta=np.ndarray>
  * time       (time) datetime64[ns] 2022-06-17T11:00:00 ... 2022-09-01
Data variables:
    H0         (lat, lon) float32 dask.array<chunksize=(371, 264), meta=np.ndarray>
    diff       (time, lat, lon) float32 dask.array<chunksize=(1, 371, 264), meta=np.ndarray>
Attributes:
    tag_id:   A18832

## define the target grid

In [7]:
%%time
grid = create_grid(nside=nside, rot=rot)
grid

CPU times: user 5.45 s, sys: 6.23 s, total: 11.7 s
Wall time: 4.73 s


HealpyGridInfo(nside=4096, rot={'lat': 0, 'lon': 30})

## compute the weights

In [8]:
%%time
regridder = HealpyRegridder(ds, grid)
regridder

CPU times: user 40 ms, sys: 16 ms, total: 56 ms
Wall time: 55.7 ms


## apply the weights

In [9]:
%%time
regridded = regridder.regrid_ds(ds)
regridded

/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 32.38 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 32.38 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


CPU times: user 5.56 s, sys: 1.98 s, total: 7.53 s
Wall time: 25.9 s


<xarray.Dataset>
Dimensions:     (time: 1814, x: 527, y: 466)
Coordinates:
  * time        (time) datetime64[ns] 2022-06-17T11:00:00 ... 2022-09-01
    latitude    (x, y) float64 dask.array<chunksize=(527, 466), meta=np.ndarray>
    longitude   (x, y) float64 dask.array<chunksize=(527, 466), meta=np.ndarray>
    cell_ids    (x, y) int64 dask.array<chunksize=(527, 466), meta=np.ndarray>
    resolution  float64 0.0002498
Dimensions without coordinates: x, y
Data variables:
    H0          (x, y) float64 dask.array<chunksize=(527, 466), meta=np.ndarray>
    diff        (time, x, y) float64 dask.array<chunksize=(1, 527, 466), meta=np.ndarray>
Attributes:
    tag_id:   A18832
    nside:    4096
    rot_lat:  0
    rot_lon:  30

## compute and save to disk

In [ ]:
regridded.to_zarr(diff_healpix_path, mode="w", consolidated=True, compute=True)

/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/distributed/client.py:3141: UserWarning: Sending large graph of size 81.19 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home1/datawork/todaka/micromamba/pangeo-fish-iaocea/lib/python3.11/site-packages/dask/core.py:121: RuntimeWa

## visualize the results

In [ ]:
regridded_ = xr.open_dataset(diff_healpix_path, engine="zarr", chunks={})
regridded_

In [ ]:
import hvplot.xarray
regridded_['diff'].isel(time=0).plot(x='longitude',y='latitude')